score_calibration
------------------------

화자의 추가 정보를 이용해서 score를 조정하여 특정 화자에 대해 성능이 좋아지게하려고 한다.



## Environment

In [2]:
%load_ext autoreload
%autoreload 2
%pylab
%matplotlib inline

import pandas as pd
import pickle
import numpy as np
import sys
import os

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [3]:
sys.path.append('/host/projects/sv_experiments/sv_system/')
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="3"

In [4]:
def key2df(keys, delimeter="-"):
    key_df = pd.DataFrame(keys, columns=['key'])
    key_df['spk'] = key_df.key.apply(lambda x: x.split(delimeter)[0])
    key_df['label'] = key_df.groupby('spk').ngroup()
    key_df['origin'] = key_df.spk.apply(lambda x: 'voxc2' if x.startswith('id') else 'voxc1')
    
    return key_df

## Local LDA  implementaion

- sv_embeds 학습 0.88%
- sv_embeds + centering 학습 0.64% (0.4 PSB)
- si_embeds로 학습했을 떄는 11.x%
- si_embeds + centering 6.628% (6.872 PSB, 6.591(imposter_mean_cosin.pkl))
- si_embeds + norm +centering 8.059% (6.998 PSB)

- si_embeds + centering 6.225% (6.166% PSB)

In [5]:
si_keys = pickle.load(open("/host/projects/sv_experiments/sv_system/gcommand_mfcc30_best/si_keys.pkl", "rb"))
si_embeds = np.load("/host/projects/sv_experiments/sv_system/gcommand_mfcc30_best/si_embeds.npy")

sv_keys = pickle.load(open("/host/projects/sv_experiments/sv_system/gcommand_mfcc30_best/sv_keys.pkl", "rb"))
sv_embeds = np.load("/host/projects/sv_experiments/sv_system/gcommand_mfcc30_best/sv_embeds.npy")
 
trial = pd.read_pickle("/dataset/SV_sets/voxceleb12/dataframes/voxc12_test_trial.pkl")
# trial = pd.read_pickle("/dataset/SV_sets/gcommand/dataframes/equal_num_30spk/equal_num_30spk_test_trial.pkl")

In [160]:
si_key_df = key2df(si_keys)
sv_key_df = key2df(sv_keys)

In [161]:
def euc_dist(a, b):
    return ((a * a) + (b * b) - b * a).sum(1)

def cos_dist(a, b):
    a = a / np.linalg.norm(a, axis=1, keepdims=True)
    b = b / np.linalg.norm(b, axis=1, keepdims=True)
    return -(a*b).sum(1)

In [162]:
si_spks = sorted(si_key_df.spk.unique().tolist())
sv_spks = sorted(sv_key_df.spk.unique().tolist())

In [164]:
## centering si_embeds
si_overall_mean = si_embeds.mean(0)
si_embeds_centered = si_embeds - si_overall_mean

mean_vectors = []
for spk in si_spks:
    mean_vectors.append(si_embeds_centered[si_key_df[si_key_df.spk == spk].index].mean(0))

# within-class scatter matrix Sw
S_W = np.zeros([512, 512])

for spk, cl_mean in zip(si_spks, mean_vectors):
    cl_embeds = si_embeds_centered[si_key_df[si_key_df.spk == spk].index]
    x_m = cl_embeds - cl_mean.reshape(1, 512)
    x_m = np.expand_dims(x_m, -1)
    cl_sc = np.matmul(x_m, x_m.transpose([0,2,1])).sum(0)
    S_W += cl_sc

# between-class scatter matrix SB
S_B = np.zeros([512, 512])

for spk, cl_mean in zip(si_spks, mean_vectors):
    ovm_m = np.expand_dims(cl_mean, -1) - np.expand_dims(si_overall_mean, -1)
    ns = len(si_key_df[si_key_df.spk == spk])
    S_B += ns*ovm_m.dot(ovm_m.T)

In [222]:
# local LDA
# modify the bewenn-class scatter matrix SB --> PSB

imposter_means = dict()

for spk, cl_mean in zip(si_spks, mean_vectors):
    cl_embeds = si_embeds_centered[si_key_df[si_key_df.spk == spk].index]
    out_embeds = si_embeds_centered[si_key_df[si_key_df.spk != spk].index]
    out_distances = cos_dist(cl_mean.reshape(1,512), out_embeds)
    max_dist_in_samples = cos_dist(cl_mean.reshape(1,512), cl_embeds).max()
    out_embeds = out_embeds[out_distances < max_dist_in_samples]
    if len(out_embeds) > 1000:
        imposter_mean = out_embeds.mean(0)
    else:
        imposter_mean = si_overall_mean
    imposter_means[spk] = imposter_mean

In [224]:
PS_B = np.zeros([512, 512])
for spk, cl_mean in zip(si_spks, mean_vectors):    
    ovm_m = cl_mean.reshape(512, 1) - imposter_means[spk].reshape(512, 1)
    ns = len(si_key_df[si_key_df.spk == spk])
    PS_B += ns*ovm_m.dot(ovm_m.T)

In [ ]:
# pickle.dump(imposter_means, open("imposter_mean.pkl", "wb"))
# imposter_means = pickle.load(open("imposter_mean_euc.pkl", "rb"))

### Eigen Analysis

In [165]:
# eig analysis
eig_vals, eig_vecs = np.linalg.eig(np.linalg.inv(S_W).dot(S_B))

In [225]:
eig_vals, eig_vecs = np.linalg.eig(np.linalg.inv(S_W).dot(PS_B))

In [226]:
# Make a list of (eigenvalue, eigenvector) tuples
eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

# Sort the (eigenvalue, eigenvector) tuples from high to low
eig_pairs = sorted(eig_pairs, key=lambda k: k[0], reverse=True)

# Visually confirm that the list is correctly sorted by decreasing eigenvalues

# print('Eigenvalues in decreasing order:\n')
# for i in eig_pairs:
#     print(i[0])

In [227]:
# print('Variance explained:\n')
# eigv_sum = sum(eig_vals)
# for i,j in enumerate(eig_pairs):
#     print('eigenvalue {0:}: {1:.2%}'.format(i+1, (j[0]/eigv_sum).real))

In [228]:
lda_dim = 50
W = np.hstack([eig_pairs[i][1].reshape(512,1) for i in range(lda_dim)])
# print('Matrix W:\n', W.real)

Matrix W:
 [[-6.08520541e-05  4.49379976e-02  8.60251520e-03 ...  2.44446541e-02
   1.36556581e-02  1.84346736e-02]
 [-4.05635188e-03 -3.77775462e-02  4.64653106e-02 ... -4.59948848e-03
   5.36177619e-03  7.11882762e-03]
 [ 2.58835289e-02  3.54241223e-02 -5.71222036e-02 ...  5.52143049e-02
  -9.07865252e-02  3.51444308e-02]
 ...
 [ 4.45784243e-03 -1.99528003e-02  4.48601447e-02 ... -1.46671021e-02
  -9.16218364e-02  4.46749959e-02]
 [ 8.52374710e-02 -2.87684122e-02 -4.83442201e-02 ...  3.79613759e-02
  -4.93495052e-02  9.75404038e-03]
 [ 7.60150856e-02  3.77620219e-03 -6.92232296e-02 ...  1.73623637e-02
  -2.76027779e-02 -4.44020240e-02]]


In [229]:
# centering sv_embeds
sv_embeds = np.load("/host/projects/sv_experiments/sv_system/gcommand_mfcc30_best/sv_embeds.npy")
sv_embeds_centered = sv_embeds - si_overall_mean

In [7]:
import torch
from torch.nn.functional import cosine_similarity

lda_sv_embeds = torch.from_numpy(sv_embeds_centered.dot(W.real))

NameError: name 'sv_embeds_centered' is not defined

In [231]:
scores = cosine_similarity(lda_sv_embeds[trial.enrolment_id], lda_sv_embeds[trial.test_id], dim=1).numpy()

In [233]:
from eval.score_utils import compute_eer

compute_eer(scores[trial.label == 1], 
            scores[trial.label == 0])

eer:11.298% at threshold 0.4893


(0.11297748123436202, 0.48931544154271034)